### Import the libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy import stats
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')

### Data Load

In [3]:
df=pd.read_csv(r"C:\Users\Praneeth\Downloads\covtype.csv\covtype.csv")
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [4]:
df.tail()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,3
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,3
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,3
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,3
581011,2383,165,13,60,4,67,231,244,141,875,...,0,0,0,0,0,0,0,0,0,3


### Exploratory Data Analysis

In [5]:
class DataFrame_Info():
    def __init__(self):
        print("Attribute information")
    
    def Attribute_information(self,df):
        """
        This method give us the basic information of all the attributes like count of attributes,number of rows & columns,numerical attributes and categorical attributes and so on
        """
        
        data_info=pd.DataFrame(columns=['No of observation',
                                       'No of variables',
                                       'No of Numerical variables',
                                       'No of Factor variables',
                                       'No of Categorival variables',
                                       'No of Logical variables',
                                       'No of Date Variables',
                                       'No of zero variance variables'])
        
        data_info.loc[0,'No of observation']=df.shape[0]

        data_info.loc[0,'No of variables']=df.shape[1]
        
        data_info.loc[0,'No of Numerical variables']=df._get_numeric_data().shape[1]
        
        data_info.loc[0,'No of Factor variables']=df.select_dtypes(include='category').shape[1]
        
        data_info.loc[0,'No of Categorical variables']=df.select_dtypes(include='object').shape[1]
        
        data_info.loc[0,'No of Logical Variables']=df.select_dtypes(include='bool').shape[1]
        
        data_info.loc[0,'No of Date Variables']=df.select_dtypes(include='datetime64').shape[1]
        
        data_info.loc[0,'No of zero variance variables']=df.loc[:,df.apply(pd.Series.nunique)==1].shape[1]
        
        data_info=data_info.transpose()
        
        data_info.columns=['value']
        
        
        
        return data_info
    
    def __get_missing_values(self,data):
        
        """
        It is a private method ,so it cannot accesed by object outsid ethe class.This function will give us a basic information like count of missing values
        """
        
        #Getting sum of missing values for each feature
        missing_values=data.isnull().sum()
        #Feature missing values are ssorted from few to many
        missing_values.sort_values(ascending=False,inplace=True)
        #get the missing values
        return missing_values
    
    def Agg_Tabulation(self,data):
        """
        This method is a extension of additional inforamtion like about the data like Entrophy value,missing value percentage and some observations
        """
        
        print('='*100)
        print('Aggrgation of Table')
        
        table=pd.DataFrame(data.dtypes,columns=['dtypes'])
        table1=pd.DataFrame(data.columns,columns=['Names'])
        table=table.reset_index()
        table=table.rename(columns={'index':'Name'})
        table['No of missing']=data.isnull().sum().values
        table['No of unique']=data.nunique().values
        table['Percent of missing']=((data.isnull().sum().values)/(data.shape[0]))*100
        table['First Observation']=data.loc[0].values
        table['Second observation']=data.loc[1].values
        table['Third Observation']=data.loc[2].values
        
    
        return table
        
        
        print('*'*100)
        print('Interquartile Range')
    def _iqr(self,x):
        """
        It is a private method return you a interquartile range
        """
            
        return x.quantile(q=0.75)-x.quantile(q=0.25)
        
    def __outlier_count(self,x):
        """
        It is a private method which return you a outliers present in a interquartile range
        """
        upper_out=x.quantile(q=0.75)+1.5 * self._iqr(x)
        lower_out=x.quantile(q=0.25)-1.5 * self._iqr(x)
            
        return len(x[x > upper_out])+len(x[x < lower_out])
    def num_count_summary(self,df):
        """
        This method which returns you basci information about Numerica variable like Positive values,Negitive Vlues,Unique Count,Zero count positive and negitive,infiniyt-count and count of outliers
        """
            
        df_num=df._get_numeric_data()
        data_info_num=pd.DataFrame()
        i=0
        for c in df_num.columns:
            data_info_num.loc[c,'Negitive values']=df_num[df_num[c] < 0].shape[0]
            data_info_num.loc[c,'Positive values']=df_num[df_num[c] > 0].shape[0]
            data_info_num.loc[c,'Zero values']=df_num[df_num[c]== 0].shape[0]
            data_info_num.loc[c,'Unique_count']=len(df_num[c].unique())
            data_info_num.loc[c,'Negitive_infinity_count']=df_num[df_num[c]== -np.inf].shape[0]
            data_info_num.loc[c,'Positive_infinity_count']=df_num[df_num[c]== np.inf].shape[0]
            data_info_num.loc[c,'Percentage of miisning values']=df_num[df_num[c].isnull()].shape[0]/df_num.shape[0]
            data_info_num.loc[c,'Count of outliers']=self.__outlier_count(df_num[c])
            i= i+1
        return data_info_num
    def statstical_summary(self,df):
        """
        This method which return you a various percentile of the data including count and mean
        """
        df_num=df._get_numeric_data()
        data_stat_num=pd.DataFrame()
            
        try:
            data_stat_num=pd.concat([df_num.describe().transpose(),
                                        pd.DataFrame(df_num.quantile(q=0.1)),
                                        pd.DataFrame(df_num.quantile(q=0.90)),
                                        pd.DataFrame(df_num.quantile(q=0.95))],axis=1)
            data_stat_num.columns=['count','mean','std','min','25%','50%','75%','max','10%','90%','95%']
        except:
            pass
        return data_stat_num

In [6]:
info=DataFrame_Info()

Attribute information


In [7]:
info.Attribute_information(df)

,value
No of observation,581012
No of variables,55
No of Numerical variables,55
No of Factor variables,0
No of Categorival variables,NaN
No of Logical variables,NaN
No of Date Variables,0
No of zero variance variables,0
No of Categorical variables,0
No of Logical Variables,0


In [8]:
info.Agg_Tabulation(df)

Aggrgation of Table


,Name,dtypes,No of missing,No of unique,Percent of missing,First Observation,Second observation,Third Observation
0,Elevation,int64,0,1978,0.0,2596,2590,2804
1,Aspect,int64,0,361,0.0,51,56,139
2,Slope,int64,0,67,0.0,3,2,9
3,Horizontal_Distance_To_Hydrology,int64,0,551,0.0,258,212,268
4,Vertical_Distance_To_Hydrology,int64,0,700,0.0,0,-6,65
5,Horizontal_Distance_To_Roadways,int64,0,5785,0.0,510,390,3180
6,Hillshade_9am,int64,0,207,0.0,221,220,234
7,Hillshade_Noon,int64,0,185,0.0,232,235,238
8,Hillshade_3pm,int64,0,255,0.0,148,151,135
9,Horizontal_Distance_To_Fire_Points,int64,0,5827,0.0,6279,6225,6121


In [9]:
info.num_count_summary(df)

,Negitive values,Positive values,Zero values,Unique_count,Negitive_infinity_count,Positive_infinity_count,Percentage of miisning values,Count of outliers
Elevation,0.0,581012.0,0.0,1978.0,0.0,0.0,0.0,15569.0
Aspect,0.0,576098.0,4914.0,361.0,0.0,0.0,0.0,0.0
Slope,0.0,580356.0,656.0,67.0,0.0,0.0,0.0,15316.0
Horizontal_Distance_To_Hydrology,0.0,556409.0,24603.0,551.0,0.0,0.0,0.0,14557.0
Vertical_Distance_To_Hydrology,55143.0,487204.0,38665.0,700.0,0.0,0.0,0.0,31463.0
Horizontal_Distance_To_Roadways,0.0,580888.0,124.0,5785.0,0.0,0.0,0.0,669.0
Hillshade_9am,0.0,580999.0,13.0,207.0,0.0,0.0,0.0,17433.0
Hillshade_Noon,0.0,581007.0,5.0,185.0,0.0,0.0,0.0,15672.0
Hillshade_3pm,0.0,579674.0,1338.0,255.0,0.0,0.0,0.0,7832.0
Horizontal_Distance_To_Fire_Points,0.0,580961.0,51.0,5827.0,0.0,0.0,0.0,31157.0


In [10]:
info.statstical_summary(df)

,count,mean,std,min,25%,50%,75%,max,10%,90%,95%
Elevation,581012.0,2959.365301,279.984734,1859.0,2809.0,2996.0,3163.0,3858.0,2581.0,3272.0,3336.0
Aspect,581012.0,155.656807,111.913721,0.0,58.0,127.0,260.0,360.0,24.0,329.0,344.0
Slope,581012.0,14.103704,7.488242,0.0,9.0,13.0,18.0,66.0,5.0,24.0,28.0
Horizontal_Distance_To_Hydrology,581012.0,269.428217,212.549356,0.0,108.0,218.0,384.0,1397.0,30.0,564.0,684.0
Vertical_Distance_To_Hydrology,581012.0,46.418855,58.295232,-173.0,7.0,30.0,69.0,601.0,0.0,121.0,165.0
Horizontal_Distance_To_Roadways,581012.0,2350.146611,1559.254870,0.0,1106.0,1997.0,3328.0,7117.0,591.0,4793.0,5483.0
Hillshade_9am,581012.0,212.146049,26.769889,0.0,198.0,218.0,231.0,254.0,176.0,241.0,246.0
Hillshade_Noon,581012.0,223.318716,19.768697,0.0,213.0,226.0,237.0,254.0,198.0,247.0,250.0
Hillshade_3pm,581012.0,142.528263,38.274529,0.0,119.0,143.0,168.0,254.0,95.0,191.0,204.0
Horizontal_Distance_To_Fire_Points,581012.0,1980.291226,1324.195210,0.0,1024.0,1710.0,2550.0,7173.0,595.0,3747.0,4944.0


In [11]:
pd.pandas.set_option('display.max_columns',None)

In [12]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


### Data Prepeocessing

In [13]:
class DataFrame_Preprocesser():
    def __init__(self):
        print('Preprocesser Object Created')
        
    def __split_numbers_char(self,row):
        head=row.rstrip('0123456789')
        tail=row[len(head):]
        return head,tail
    def reverse_one_hot_encode(self,dataframe,start_loc,end_loc,numeric_col_name):
        dataframe['String_Column']=(dataframe.iloc[:, start_loc:end_loc]==1).idxmax(1)
        dataframe['Tuple_Column']=dataframe['String_Column'].apply(self.__split_numbers_char)
        dataframe[numeric_col_name]=pd.to_numeric(dataframe['Tuple_Column'].apply(lambda x:x[1]),errors='coerce')
        dataframe.drop(columns=['String_Column','Tuple_Column'],inplace=True)

In [14]:
preprocesser=DataFrame_Preprocesser()

Preprocesser Object Created


In [15]:
preprocesser.reverse_one_hot_encode(df,14,54,'soil_type')

In [16]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type,soil_type
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,29
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,29
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,12
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,30
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,29


In [17]:
preprocesser.reverse_one_hot_encode(df,10,14,'wilderness_area')

In [18]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type,soil_type,wilderness_area
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,29,1
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,29,1
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,12,1
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,30,1
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,29,1


In [19]:
df.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40

In [20]:
col=['Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40']
col_1=['Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4']

class col_dopper():
    def __init__(self):
        print('Column Dropped object is created')
        
    def dropper(self,x):
        """
        This method which help us to drop the column in our original dataframe which is available in the col_list and returns us final_dataset
        """
        
        data=[]
        for i in x.columns:
            if i not in col:
                data.append(i)
        return x[data]

In [21]:
column_dropper=col_dopper()

Column Dropped object is created


In [22]:
df=column_dropper.dropper(df)

In [23]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Cover_Type,soil_type,wilderness_area
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,5,29,1
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,5,29,1
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,2,12,1
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,2,30,1
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,5,29,1


### Feature Engineering

In [24]:
class Make_feature_enginnering():
    def __init__(self):
        print('Feature Enginnering object created')
    def feature_for_train(self,df):
        df['Hydro_Fire']=df['Horizontal_Distance_To_Hydrology']+df['Horizontal_Distance_To_Fire_Points']
        df['Road_Fire']=df['Horizontal_Distance_To_Roadways']+df['Horizontal_Distance_To_Fire_Points']
        df['Hydro_Road']=df['Horizontal_Distance_To_Hydrology']+df['Horizontal_Distance_To_Roadways']
        df['Hydro_Road_Sub']=np.abs(df['Horizontal_Distance_To_Roadways']-df['Horizontal_Distance_To_Hydrology'])
        df['Hydro_Fire_Sub']=np.abs(df['Horizontal_Distance_To_Fire_Points']-df['Horizontal_Distance_To_Hydrology'])
        df['Road_Fire_Sub']=np.abs(df['Horizontal_Distance_To_Fire_Points']-df['Horizontal_Distance_To_Roadways'])
        df['RAD_Slope']=df['Slope'].apply(lambda x:x*(np.pi/180))
        df['RAD_Aspect']=df['Aspect'].apply(lambda x:x*(np.pi/180))
        df['ELEV_DIS']=df['Elevation']-df['Horizontal_Distance_To_Hydrology']*0.2
        df['ELEV_FIRE']=df['Elevation']-df['Horizontal_Distance_To_Fire_Points']*0.2
        df['ELEV_ROAD']=df['Elevation']-df['Horizontal_Distance_To_Roadways']*0.2

In [25]:
FE=Make_feature_enginnering()

Feature Enginnering object created


In [26]:
FE.feature_for_train(df)

In [27]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Cover_Type,soil_type,wilderness_area,Hydro_Fire,Road_Fire,Hydro_Road,Hydro_Road_Sub,Hydro_Fire_Sub,Road_Fire_Sub,RAD_Slope,RAD_Aspect,ELEV_DIS,ELEV_FIRE,ELEV_ROAD
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,5,29,1,6537,6789,768,252,6021,5769,0.052360,0.890118,2544.4,1340.2,2494.0
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,5,29,1,6437,6615,602,178,6013,5835,0.034907,0.977384,2547.6,1345.0,2512.0
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,2,12,1,6389,9301,3448,2912,5853,2941,0.157080,2.426008,2750.4,1579.8,2168.0
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,2,30,1,6453,9301,3332,2848,5969,3121,0.314159,2.705260,2736.6,1542.8,2167.0
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,5,29,1,6325,6563,544,238,6019,5781,0.034907,0.785398,2564.4,1360.6,2516.8


### Missing value imputation

In [28]:
class Missing_value_handle():
    def __init__(self):
        print('Mssing value Object is created')
    def knn_imputer(self,df):
        """
        This method which helps to impute the missing values in the attributes by the KNN imputer
        """
        knn_imputer=KNNImputer(n_neighbors=5)
        df.iloc[:, : ]=knn_imputer.fit_transform(df)
        return df

In [29]:
imputer=Missing_value_handle()

Mssing value Object is created


In [30]:
df=imputer.knn_imputer(df)

In [31]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Cover_Type,soil_type,wilderness_area,Hydro_Fire,Road_Fire,Hydro_Road,Hydro_Road_Sub,Hydro_Fire_Sub,Road_Fire_Sub,RAD_Slope,RAD_Aspect,ELEV_DIS,ELEV_FIRE,ELEV_ROAD
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,1.0,0.0,0.0,0.0,5.0,29.0,1.0,6537.0,6789.0,768.0,252.0,6021.0,5769.0,0.052360,0.890118,2544.4,1340.2,2494.0
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,1.0,0.0,0.0,0.0,5.0,29.0,1.0,6437.0,6615.0,602.0,178.0,6013.0,5835.0,0.034907,0.977384,2547.6,1345.0,2512.0
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,1.0,0.0,0.0,0.0,2.0,12.0,1.0,6389.0,9301.0,3448.0,2912.0,5853.0,2941.0,0.157080,2.426008,2750.4,1579.8,2168.0
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,1.0,0.0,0.0,0.0,2.0,30.0,1.0,6453.0,9301.0,3332.0,2848.0,5969.0,3121.0,0.314159,2.705260,2736.6,1542.8,2167.0
4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,6172.0,1.0,0.0,0.0,0.0,5.0,29.0,1.0,6325.0,6563.0,544.0,238.0,6019.0,5781.0,0.034907,0.785398,2564.4,1360.6,2516.8


### Model selection

In [32]:
from sklearn.model_selection import cross_val_score,KFold,RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [33]:
x=df.drop('Cover_Type',axis=1)
y=df['Cover_Type']

In [34]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [36]:
models=Model_Selector()

Model Selector object created


In [35]:
class Model_Selector():
    
    

    def __init__(self):
        print("Model Selector object created")
        
    """
    This method helps to select
    the best machine learning 
    model to compute the relationship
    betweem i/p and d/p variable
    
    """    
        
        
    def Classification_Model_Selector(self,df):
        seed = 42
        models = []
        models.append(("LR", LogisticRegression()))
        models.append(("RF", RandomForestClassifier(n_estimators=100,\
            random_state=42,max_depth=10)))
        models.append(("KNN", KNeighborsClassifier()))
        models.append(("CART", DecisionTreeClassifier()))
        models.append(("XGB", XGBClassifier()))
        result = []
        names = []
        scoring = 'accuracy'
        seed = 42
        
        

        for name, model in models:
            kfold = KFold(n_splits = 5, random_state =seed)
            cv_results = cross_val_score(model, x_train,\
                    y_train, cv = kfold, scoring = scoring)
            result.append(cv_results)
            names.append(name)
            msg = (name, cv_results.mean(), cv_results.std())
            print(msg)
            
            
            
        fig = plt.figure(figsize = (8,4))
        fig.suptitle('Algorithm Comparison')
        ax = fig.add_subplot(1,1,1)
        plt.boxplot(result)
        ax.set_xticklabels(names)
        plt.show()

In [ ]:
models.Classification_Model_Selector(df)

('LR', 0.6095085504582876, 0.0020090083521059413)
('RF', 0.81633997916954, 0.0011599700106204802)
('KNN', 0.9536350420881993, 0.0009004263712174171)
('CART', 0.9406281654420573, 0.0004656756249015674)
